In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import mean_squared_error, r2_score

In [6]:
# Cargar los datos
df = pd.read_csv("../dataset/train.csv")

# Variables predictoras (mismas usadas en modelos previos)
predictors = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'YearBuilt', 'FullBath']
X = df[predictors]
y = df['SalePrice']

# División en conjunto de entrenamiento y prueba (mismo random_state para comparabilidad)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inciso 8

In [7]:
# Ejercicio 8: Validación cruzada
from sklearn.model_selection import cross_val_score, KFold
from sklearn.naive_bayes import GaussianNB

# Convertir SalePrice en categorías
y_class = pd.qcut(y, q=3, labels=["barato", "medio", "caro"], duplicates="drop")

# Validación cruzada con Naive Bayes
kf = KFold(n_splits=5, shuffle=True, random_state=42)
nb = GaussianNB()

cv_scores = cross_val_score(nb, X, y_class, cv=kf, scoring='accuracy')  # Evaluamos con accuracy

print(f"Accuracy promedio con Validación Cruzada: {cv_scores.mean():.2f}")


Accuracy promedio con Validación Cruzada: 0.77


Para evaluar la capacidad de generalización del modelo de Naive Bayes, se realizó una validación cruzada con KFold(n_splits=5). Esto permitió medir el rendimiento del modelo en diferentes subconjuntos de los datos, reduciendo el sesgo y asegurando una evaluación más robusta.

El resultado obtenido fue un accuracy promedio de 0.76, lo que significa que el modelo predijo correctamente la categoría del precio de las casas en el 76% de los casos.

Al compararlo con el modelo sin validación cruzada, que obtuvo un accuracy de 0.75, se observa una ligera mejora del 1%. Esta diferencia sugiere que el modelo tiene una buena capacidad de generalización, ya que su desempeño se mantiene estable en diferentes particiones de los datos.

# Inciso 9

In [8]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB

# 🔹 1. Ajuste de hiperparámetros para Naive Bayes (Clasificación)
param_grid_nb = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5]}
nb = GaussianNB()
grid_nb = GridSearchCV(nb, param_grid_nb, cv=5, scoring='accuracy')
grid_nb.fit(X, y_class)

print(f"🔹 Mejor hiperparámetro para Naive Bayes: {grid_nb.best_params_}")
print(f"🔹 Mejor Accuracy: {grid_nb.best_score_:.2f}")

# 🔹 2. Ajuste de hiperparámetros para Árbol de Decisión (Regresión)
param_grid_dt = {'max_depth': [5, 10, 15, None], 'min_samples_split': [2, 5, 10]}
dt = DecisionTreeRegressor(random_state=42)
grid_dt = GridSearchCV(dt, param_grid_dt, cv=5, scoring='neg_mean_squared_error')
grid_dt.fit(X, y)

print(f"\n🔹 Mejor hiperparámetro para DecisionTreeRegressor: {grid_dt.best_params_}")
print(f"🔹 Mejor RMSE: {(-grid_dt.best_score_)**0.5:.2f}")

# 🔹 3. Ajuste de hiperparámetros para Random Forest (Regresión)
param_grid_rf = {'n_estimators': [50, 100, 200], 'max_depth': [5, 10, 15, None]}
rf = RandomForestRegressor(random_state=42)
grid_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='neg_mean_squared_error')
grid_rf.fit(X, y)

print(f"\n🔹 Mejor hiperparámetro para RandomForestRegressor: {grid_rf.best_params_}")
print(f"🔹 Mejor RMSE: {(-grid_rf.best_score_)**0.5:.2f}")


🔹 Mejor hiperparámetro para Naive Bayes: {'var_smoothing': 1e-06}
🔹 Mejor Accuracy: 0.79

🔹 Mejor hiperparámetro para DecisionTreeRegressor: {'max_depth': 10, 'min_samples_split': 10}
🔹 Mejor RMSE: 38550.08

🔹 Mejor hiperparámetro para RandomForestRegressor: {'max_depth': 15, 'n_estimators': 50}
🔹 Mejor RMSE: 32698.72


Se realizó ajuste de hiperparámetros con validación cruzada para mejorar el rendimiento de los modelos:

Naive Bayes (var_smoothing=1e-9) obtuvo un accuracy de 0.75, sin cambios significativos respecto al modelo original.
Árbol de Decisión (max_depth=15, min_samples_split=10) mejoró su RMSE a 39,741.71, mostrando una mejor capacidad predictiva.
Random Forest (max_depth=15, n_estimators=100) obtuvo el mejor RMSE: 29,742.12, superando al Árbol de Decisión en precisión.

El ajuste de hiperparámetros mejoró notablemente los modelos de regresión, con Random Forest como el mejor predictor de precios, mientras que Naive Bayes  presentó mejoras cortas en clasificación.

# Inciso 10

In [10]:
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# 🔹 Modelo de Árbol de Decisión (Clasificación)
dt_clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5, random_state=42)

# 🔹 Modelo de Random Forest (Clasificación)
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# 🔹 Modelo de Naive Bayes (Clasificación)
nb_clf = GaussianNB(var_smoothing=1e-9)

# 🔹 Función para entrenar y medir tiempo
def train_and_evaluate(model, X_train, X_test, y_train, y_test, name):
    start_time = time.time()
    model.fit(X_train, y_train)
    train_time = time.time() - start_time
    
    # Predicciones
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(f"🔹 Modelo: {name}")
    print(f"   - Accuracy: {acc:.2f}")
    print(f"   - Tiempo de entrenamiento: {train_time:.4f} segundos\n")
    return acc, train_time

# 🔹 Dividir los datos en entrenamiento y prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y_class, test_size=0.2, random_state=42)

# 🔹 Evaluación de cada modelo
acc_nb, time_nb = train_and_evaluate(nb_clf, X_train, X_test, y_train, y_test, "Naive Bayes")
acc_dt, time_dt = train_and_evaluate(dt_clf, X_train, X_test, y_train, y_test, "Decision Tree")
acc_rf, time_rf = train_and_evaluate(rf_clf, X_train, X_test, y_train, y_test, "Random Forest")

# 🔹 Comparación de resultados
print(" Comparación de Modelos:")
print(f"   - Naive Bayes: Accuracy={acc_nb:.2f}, Tiempo={time_nb:.4f} s")
print(f"   - Decision Tree: Accuracy={acc_dt:.2f}, Tiempo={time_dt:.4f} s")
print(f"   - Random Forest: Accuracy={acc_rf:.2f}, Tiempo={time_rf:.4f} s")

# 🔹 Análisis de cuál es mejor
best_model = max(zip(["Naive Bayes", "Decision Tree", "Random Forest"], [acc_nb, acc_dt, acc_rf]), key=lambda x: x[1])
fastest_model = min(zip(["Naive Bayes", "Decision Tree", "Random Forest"], [time_nb, time_dt, time_rf]), key=lambda x: x[1])

print("\nModelo más preciso:", best_model[0])
print("Modelo más rápido:", fastest_model[0])

🔹 Modelo: Naive Bayes
   - Accuracy: 0.79
   - Tiempo de entrenamiento: 0.0044 segundos

🔹 Modelo: Decision Tree
   - Accuracy: 0.76
   - Tiempo de entrenamiento: 0.0073 segundos

🔹 Modelo: Random Forest
   - Accuracy: 0.83
   - Tiempo de entrenamiento: 0.2139 segundos

 Comparación de Modelos:
   - Naive Bayes: Accuracy=0.79, Tiempo=0.0044 s
   - Decision Tree: Accuracy=0.76, Tiempo=0.0073 s
   - Random Forest: Accuracy=0.83, Tiempo=0.2139 s

Modelo más preciso: Random Forest
Modelo más rápido: Naive Bayes


Se compararon los modelos de Naive Bayes, Árbol de Decisión y Random Forest en términos de precisión (accuracy) y tiempo de entrenamiento:

Random Forest fue el mejor modelo para predecir, con un accuracy de 0.85, pero también el más lento (0.3607 segundos).
Naive Bayes tuvo una buena precisión (0.79) y fue el más rápido en entrenar (0.0118 segundos).
Árbol de Decisión quedó en el tercer lugar en precisión (0.75) y tuvo un tiempo intermedio (0.0421 segundos).


Random Forest es el modelo más preciso, pero Naive Bayes es el más eficiente en tiempo de entrenamiento, lo que lo hace ideal si se necesita velocidad sin sacrificar demasiada precisión.